# ex-41 port lmoments 

In [1]:
import numpy as np
import scipy.special as sp
from numba import njit
import lmoments3 as lm3
from lmoments3 import distr
from typing import List
from scipy.stats import genextreme

In [2]:
c   = -0.1
loc = 0.5

r  = genextreme.rvs(c, size=1000)
r2 = genextreme.rvs(0, size=1000)

In [17]:
@njit
def comb(N, k, exact=1):
    if exact:
        if (k > N) or (N < 0) or (k < 0):
            return 0
        val = 1
        for j in np.arange(min(k, N-k)):
            val = (val*(N-j))//(j+1)
        return val

    
@njit
def samlmu(x, nmom=5):
    x = sorted(x)
    n = len(x)   
    ##Calculate first order, pretty efficient, no loops
    coefl1 = 1.0/comb(n,1)
    suml1 = sum(x)
    l1 = coefl1*suml1

    ##Calculate Second order

    #comb terms appear elsewhere, this will decrease calc time
    #for nmom > 2, and shouldn't decrease time for nmom == 2
    #comb1 = comb(i-1,1)
    #comb2 = comb(n-i,1)
    comb1 = []
    comb2 = []
    for i in range(1,n+1):
        comb1.append(comb(i-1,1))
        comb2.append(comb(n-i,1))
    
    coefl2 = 0.5 * 1.0/comb(n,2)
    xtrans = []
    for i in range(1, n+1):
        coeftemp = comb1[i-1]-comb2[i-1]
        xtrans.append(coeftemp*x[i-1])
    
    l2 = coefl2 * sum(xtrans)

    ##Calculate Third order
    #comb terms appear elsewhere, this will decrease calc time
    #for nmom > 2, and shouldn't decrease time for nmom == 2
    #comb3 = comb(i-1,2)
    #comb4 = comb(n-i,2)
    comb3 = []
    comb4 = []
    for i in range(1, n+1):
        comb3.append(comb(i-1,2))
        comb4.append(comb(n-i,2))
    
    coefl3 = 1.0/3 * 1.0/comb(n,3)
    xtrans = []
    for i in range(1, n+1):
        coeftemp = (comb3[i-1]-
                    2*comb1[i-1]*comb2[i-1] +
                    comb4[i-1])
        xtrans.append(coeftemp*x[i-1])

    l3 = coefl3 *sum(xtrans) /l2

    ##Calculate Fourth order
    #comb5 = comb(i-1,3)
    #comb6 = comb(n-i,3)
    comb5 = []
    comb6 = []
    for i in range(1,n+1):
        comb5.append(comb(i-1,3))
        comb6.append(comb(n-i,3))
    
    coefl4 = 1.0/4 * 1.0/comb(n,4)
    xtrans = []
    for i in range(1, n+1):
        coeftemp = (comb5[i-1]-
                    3*comb3[i-1]*comb2[i-1] +
                    3*comb1[i-1]*comb4[i-1] -
                    comb6[i-1])
        xtrans.append(coeftemp*x[i-1])

    l4 = coefl4 *sum(xtrans)/l2

    ##Calculate Fifth order
    coefl5 = 1.0/5 * 1.0/comb(n,5)
    xtrans = []
    for i in range(1, n+1):
        coeftemp = (comb(i-1,4)-
                    4*comb5[i-1]*comb2[i-1] +
                    6*comb3[i-1]*comb4[i-1] -
                    4*comb1[i-1]*comb6[i-1] +
                    comb(n-i,4))
        xtrans.append(coeftemp*x[i-1])

    l5 = coefl5 *sum(xtrans)/l2

    # return 5 moments
    ret = np.array([l1, l2, l3, l4, l5])
    return ret


@njit
def pelgum(xmom):
    eu = 0.577215664901532861
    if xmom[1] <= 0:        
        return None
    else:
        para2 = xmom[1] / np.log(2)
        para1 = xmom[0] - eu*para2
        para = np.array([para1, para2])
        return para
    

@njit   
def quagum(prob: np.array, para):
    u = para[0]
    a = para[1]
    #prob = np.array(prob)
   
    val = u - a * np.log(-np.log(prob))
    return val

print(comb(25., 2))
sp.comb(25, 2)

%%time
samlmu(r, 5)

%%time
lm3.lmom_ratios(r, 5)

In [18]:
momx = samlmu(r2, 5)
momx

array([0.57874939, 0.69587498, 0.15926436, 0.15212441, 0.05720893])

In [19]:
probs = [0.5, 0.8, 0.9, 0.95, 0.98, 0.99, 0.995, 0.9999]

In [25]:
para_gum = pelgum(momx)
%timeit quagum(np.array(probs), para_gum)

2.47 µs ± 46.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [26]:
para_ = distr.gum.lmom_fit(r2)
model = distr.gum(**para_)
%timeit model.ppf(probs)

84.4 µs ± 287 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
